 ## Correos Electronicos SPAM: Un enfoque con procesamiento de lenguaje Natural

### Cruz David Hernandez Antunez 15111430

Los correos electronicos no deseados en su bandeja de entrada son molestos ya que perturban la rutina del usuario. Es por eso que las cuentas de correo electronico ya tienen un filtro de spam. Dado que es una de las aplicaciones del PLN mas utilizadas vamos a ver como se desarrolla un filtro de spam simple para correos electronicos.

In [1]:
#importamos las famosas librerias
from functools import reduce
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import string
import re

In [2]:
#insertand los datos
full_corpus = pd.read_csv('SMSSpamCollection.tsv', sep='\t', header= None, names=['label', 'msg_body'])

# separando los mensajes en 'ham' y 'spam'
ham_text=[]
spam_text= []

### Bigrams

Los N-grammos se usan para modelar el lenguaje en función de la prediccion de palabra, es decir, predice la siguiente palabra de una oracion de palabras N-1 anteriores. Biagrams en la secuencia de 2 palabras de N-gramos que predice la siguiente palabra de una oracion usando la palabra anterior. En lugar de considerar la historia completa de una oracion o una secuencia de palabra en particular, un modelo como bigram puede ser ocupado en terminos de una aproximacion de la historia al ocupar una historia limitada.


La identificacion de un mensaje como "ham" o "spam" es una tarea de clasificacion ya que la variable de destino tiene valores discretos que son "ham" o "spam". En esta practica se usa el modelo bigram, aunque existen muchas tecnicas avanzadas que se pueden utilizar para este proposito. Para utilizar el modelo bigram para asignar un mensaje dado como "spam o "ham". Hay varios pasos que deben lograrse

### 1.- Inspección y separacion de mensajes en las catergorais "ham" y "spam"

Inicialmente, el conjunto de datos se debe inspeccionar para ocuparlo y abordarlo para lograr la tarea. El formato de los datos dados, la cantidad de datos proporcionados, la naturaleza de los datos se incluyen en esta inspeccion para identificar la mejor aproximacion posible para la tarea.

El corpus de mensajes dado ha marcado cada mensaje como ham o spam. Ademas hay 5568 mensajes en un DataFrame escrito en ingles que no son objets nulos. Por lo tanto, el archivo tsv se puede leer usando DataFrame en python para clasificar esos mensajes de acuerdo con el indicador dado.


In [3]:
def separate_msgs():
    for index, column in full_corpus.iterrows():
        label = column[0]
        message_text= column[1]
        if label=='ham':
            ham:text.append(message_text)
        elif label=='spam':
            spam_text.append(message_text)
separate_msgs()

### 2.- Preprocesamiento de texto

El preprocesamiento es la tarea de realizar los pasos de preparacion en el corpus de texto sin formato para completar de manera eficiente una extracción de texto o procesamiento de lenguaje natural o cualquier otra tarea que incluya texto sin formato. El preprocesamiento de texto consta de varios pasos, aunque algunos d ellos pueden no aplicarse a una tarea en particular debido a la naturaleza del conjunto de datos disponible.

En esta tarea, el preprocesamiento de texto incluye los siguientes pasos de acuerdo al conjunto de datos: 


### Eliminacion de signos de puntuacion

In [6]:
# Eliminacion de los signos de puntuacion de los mensajes de correro electronico
def remove_msg_punctuations(email_msg):
    puntuation_removed_msg = "".join([word for word in email_msg if word not in string.punctuation])
    return puntuation_removed_msg

### Convertir a minusculas:

Convertir a minusculas: La conversacion de todos los caracteres de texto en un contexto común, como los soportes en minúsculas, impide identificar dos palabras de manera diferente donde una está en minuscula y la otra no. Por ejemplo, "Primero" y "primero" deben identificarse iguales. Por lo tanto poner en minuscula todos los caracteres facilita la tarea. Ademas, Las palabras de detencion tambien esta en minusculas, Por lo que esto también haría posble eliminar palabras de detencion mas adelante.

### Tokenizing: 

Tokenizing: La tokenizacion es la tarea de dividir el texto en partes significativas, es decir, tokens que incluyen oraciones y palabras. un token se puede considerar como una instancia de una secuenta de caracteres en un texto particular que se agrupa para proporcionar una unidad semantica util para su posterior procesamiento. En esta tarea, la tokenizacion de palabras se realiza combinando espacios en blanco entre palabras como delimitador. Esto se logra en Python usando expresiones regulares para dividir una cadena en subcadenas con la funcion split(), que es un tokenizador basico

In [7]:
# Convierte el texto en minusculo y tokenizing de palabras
def tokenize_into_words(text):
    tokens=re.split('\W+', text)
    return tokens

### Palabras lematizantes: 

La derivacion es el proceso de eliminar afijos(sufijos, prefijos, infijos,circunfijos) de una palabra para obtener su raiz de palabra. Aunque la lematizacion esta relacionada con la derivacion, difiere ya que la lematizacion puede capturar formas canonicas basadas en el lema de una palabra. La lematizacion ocupa un vocabulario y un analisis morgologico de las palabras que lo hacen mas rapido y preciso que la derivacion. WordNetLemmatizer ha logrado la lematizacion en lenguaje python.

In [16]:
# lematizing
word_lemmatizer= WordNetLemmatizer ()
def lemmatization(tokenized_words):
    lemmatized_text= [word_lemmatizer.lemmatize(word) for word in tokenized_words]
    return ''.jouin(lemmatized_text)
def preprocessing_msgs(corpus):
    categorized_text = pd.DataFrame(corpus)
    categorized_text['non_punc_message_body'] = categorized_text[0].apply(lambda msg: remove_msg_punctuations(msg))
    categorized_text['tokenized_msg_body'] = categorized_text['non_punc_message_body'].apply(lambda msg: tokenize_into_words(msg.lower()))
    categorized_text['lemmatized_msg_words'] = categorized_text['tokenized_msg_body'].apply(lambda word_list: lemmatization(word_list))
    return categorized_text['lemmatized_msg_words']

### 3.- Extracción de características

Después de la etapa de preprocesamiento, las caracteristicas deben extraerse del texto. Las caracteristicas son las unidades que admiten la tarea de clasificacion, y las bigrams son las caracteristicas en esta tarea de clasificacion de mensajes. Los bigrams o las caracteristicas se extraen del texto preprocesado. inicialmente, los unigramas se adquieren y luego esos unigramas se utilizan para obtener los unigramas en cada corpus ("ham" y "spa,")

In [9]:
# Extraccion de caracteristicas. Ejemplo: n-grams
def feature_extraction(preprocessed_text):
    bigrams=[]
    unigrams_lists=[]
    for msg in preprocessed_text:
        #agregando end of and start of al mensaje
        msg='<s>'+ msg+'</s>'
        unigrams_lists.append(msg.split())
    unigrams = [uni_list for sub_list in unigrams_lists for uni_list in sub_list]
    bigrams.extend(nltk.bigrams(unigrams))
    return bigrams

### 4. Eliminacion de Stop Words

Hay ciertas palabras en un idioma (se utiliza inglés en la practica) que son necesarias para una oracion o una secuencia de palabras, aunque no cintribuyen al significado de una frase considerada. la biblioteca del kit de herramientas de lenguaje natural (NLTK) en python proporciona palabras de detencion comunes para algunos idiomas.


En lugar de eliminar las palabras de detencion en el paso de preprocesamiento, se realiza despues de extraer las caracteristicas del corpus para evitar la ausencia de bigrams con palabras de una parada (("use","your"),("to","win")) al adiquierir las funciones, ya que tienen un impacto en el resultado final de la aplicacion. Las palabras de detencion se pueden ignorar en esta recuperacion de informacion orientada a palabras clave debido a su efecto en la precision de la recuperacion

In [10]:
# Eliminando bigrams solo con stop words
stopwords=nltk.corpus.stopwords.words('english')
def filter_stopwords_bigrams(bigram_list):
    filtered_bigrams=[]
    for bigram in bigram_list:
        if bigram[0] in stopwords and bigram[1] in stopwords:
            continue
        filtered_bigrams.appends(bigram)
        return filtered_bigrams

### 5.- Obtener distribucion de frecuencia de caracteristicas

La distribucion de frecuencia se utiliza para obtener la frecuencia de aparicion de cada elemento de vocabulario en un texto determinado.

In [18]:
# Adquiriendo la frecuencia de caracteristicas
def ham_bigram_feature_frequency():
    # frecuencia de caracteristicas para ensajes ham
    ham_bigrams=feature_extraction(preprocessing_msgs(ham_text))
    ham_bigram_frequency = nltk.FreqDist(filter_stopwords_bigrams(ham_bigrams))
    return ham_bigram_frequency
def spam_bigram_feature_frequency():
    # frecuencia de caracteristicas para mensaje spam
    spam_bigrams= feature_extraction(preprocessing_msgs(spam_text))
    spam_bigram_frequency = nltk.FreqDist(filter_stopwords_bigrams(spam_bigrams))
    return spam_bigram_frequency


### 6.- Construyendo un modelo para la predicción

El modelo para clasificar un mensaje dado como "ham" o "spam" se ha abordado calculando las probabilidads de bigram dentro de cada corpus. Inicialmente, el mensaje dado debe procesarse previamente para avanzar con clasificacion, incluida la eliminacion de signos de puntuacion, el cambio de todos los caracteres a minusculas, la tokenizacion y la lematizacion. Luego los bigrams se extraen del texto preprocesado para calcular finalmente la probabilidad de que el texto esté en cada corpus "ham" o "spam"

In [17]:
# Calculando probabilidades del bigram
def bigram_probability(message):
    probability_h = 1
    probability_s = 1
    # Preprocesando los mensaje de entrada
    punc_removed_message = "".join(word for word in message if word not in string.punctuation)
    punc_removed_message = '<s> ' +punc_removed_message +' </s>'
    tokenized_msg = re.split('\s+', punc_removed_message)
    lemmatized_msg = [word_lemmatizer.lemmatize(word)for word in tokenized_msg]
    # bigrams para el mensaje
    bigrams_for_msg = list(nltk.bigrams(lemmatized_msg))
    # Eliminamos stop words
    ham_unigrams = [word for word in feature_extraction(preprocessing_msgs(ham_text)) if word not in stopwords]
    spam_unigrams = [word for word in feature_extraction(preprocessing_msgs(spam_text)) if word not in stopwords]
    # Frecuencias de bigrams extraidas
    ham_frequency = ham_bigram_feature_frequency()
    spam_frequency  = spam_bigram_feature_frequency()
    print('========================== Calculando Probabilidades ==========================')
    print('----------- Frecuencias Ham ------------')
    for bigram in bigrams_for_msg:
        # probabilidad de la primera palabra en bigram
        ham_probability_denominator = 0
        # probabilidad de bigram (suavizado) 
        ham_probability_of_bigram = ham_frequency[bigram] + 1
        print(bigram, ' ocurre ', ham_probability_of_bigram)
        for (first_unigram, second_unigram) in filter_stopwords_bigrams(ham_unigrams):
            ham_probability_denominator += 1
            if(first_unigram == bigram[0]):
                ham_probability_denominator += ham_frequency[first_unigram, second_unigram]
        probability = ham_probability_of_bigram / ham_probability_denominator
        probability_h *= probability
    print('\n')
    print('----------- Frecuencias Spam ------------')
    for bigram in bigrams_for_msg:
        # probabilidad de la primera palabra en bigram
        spam_probability_denominator = 0
        # probabilidad de bigram (suavizado) 
        spam_probability_of_bigram = spam_frequency[bigram] + 1
        print(bigram, ' ocurre ', spam_probability_of_bigram)
        for (first_unigram, second_unigram) in filter_stopwords_bigrams(spam_unigrams):
            spam_probability_denominator += 1
            if(first_unigram == bigram[0]):
                spam_probability_denominator += spam_frequency[first_unigram, second_unigram]
        probability = spam_probability_of_bigram / spam_probability_denominator
        probability_s *= probability
    print('\n')
    print('Probabilidad Ham: ' +str(probability_h))
    print('Probabildiad Spam: ' +str(probability_s))
    print('\n')
    if(probability_h >= probability_s):
        print('\"' +message +'\" es un mensaje Ham')
    else:
        print('\"' +message +'\" es un mensaje Spam')
    print('\n')
bigram_probability('Click here,  ..to win an iphone 11 pro max')
bigram_probability('Win a brand new car ')

KeyError: 0